In [ ]:
#pip install ragas

import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    AnswerRelevancy,      # 답변 관련성
    Faithfulness,        # 답변 충실도 (hallucination 방지)
    ContextRelevance,    # 검색된 문서 관련성
    ContextRecall,       # 검색된 문서 재현율
    ContextPrecision     # 검색된 문서 정밀도
)
from datasets import Dataset

# 1. 평가용 데이터셋 생성
def create_evaluation_dataset():
    
    # 평가용 질문-답변-컨텍스트 데이터 (각 리스트 길이를 맞춰줌)
    eval_data = {
        "question": ["""
                     DB손해보험 운전자보험은 중도에 해지하면 해약환급금을 얼마나 받을 수 있나요?
                     """ #위의 질문 활용
        ],
        "answer": ["""
                   DB손해보험 운전자보험의 **해약환급금**에 대한 정보는 다음과 같습니다:
                   **해약환급금 지급 여부**: DB손해보험의 운전자보험은 중도 해지 시 **해약환급금이 없습니다**.
                   **해약환급금 산정 방식**:
                   해약환급금은 납입한 보험료에서 계약 체결 및 유지 관리에 소요되는 경비와 경과된 기간의 위험 보장에 사용된 보험료를 차감하여 지급됩니다.
                   따라서, 중도 해지 시에는 **납입한 보험료보다 적거나 아예 지급되지 않을 수 있습니다**.
                   ### 요약
                   | 항목               | 내용                                   |
                   |------------------|--------------------------------------|
                   | 해약환급금 지급 여부 | **없음**                               |
                   | 해약환급금 산정 방식 | 납입한 보험료에서 경비 및 위험 보장 차감 |
                   이와 같은 조건을 고려할 때, DB손해보험 운전자보험은 중도 해지 시 해약환급금이 없으므로, 장기적으로 유지할 계획이 아니라면 신중하게 결정해야 합니다.
                   더 궁금한 점이 있으신가요?
                   """
                   ],
        
        "contexts": [
            [
                """
                계약자가 보험계약을 중도에 해지할 경우 보험회사는 해약환급금을 지급합니다.
                ① 해약환급금은 납입한 보험료 보다 적거나 없을 수도 있습니다.
                *해약환급금: 납입한 보험료에서 계약체결·유지관리 등에 소요되는 경비 및 경과된 기간의 위험 보장에 사용된 보험료를 차감하여 지급
                ② 납입기간 중 보험계약을 해지할 경우 해약환급금이 없습니다. (해약환급금 미지급형)
                ③ 납입기간 중 보험계약을 해지할 경우 표준형 보다 해약환급금이 적습니다. (해약환급금 저지급형)
                """
            ],
        ],
        "ground_truth": [
            """
            DB손해보험 운전자보험을 중도에 해지하면 해약환급금은 납입한 보험료 보다 적거나 없을 수도 있습니다.
            구체적인 계산 방식이나 금액에 대한 정보가 포함되어 있지 않습니다. 따라서 정확한 해약환급금은 가입하신 보험의 약관이나 실제 납입 금액, 해지 시점 등을 확인해야만 알 수 있습니다.
            """
            ]
    }
    
    return Dataset.from_dict(eval_data)

# 2. 평가 결과 분석
def analyze_evaluation_results(result):
    """RAGAS 평가 결과 분석 및 출력"""
    
    print("=== RAG 성능 평가 결과 ===")
    
    try:
        df = result.to_pandas()
        print("평가 결과:")
        print(df)
        
        print("\n=== 메트릭별 평균 점수 ===")
        numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_columns:
            avg_score = df[col].mean()
            print(f"{col}: {avg_score:.3f}")
            
    except Exception as e:
        print(f"결과 분석 중 오류: {e}")
        print("결과 객체 타입:", type(result))
        print("결과 내용:", result)


# 실행 코드
if __name__ == "__main__":
    try:
        # 1. 평가 데이터셋 생성
        dataset = create_evaluation_dataset()
        
        # 2. RAGAS 평가 실행
        result = evaluate(
            dataset=dataset,
            metrics=[
                Faithfulness(),
                AnswerRelevancy(),
                ContextRelevance(),
                ContextRecall(),
                ContextPrecision()
            ]
        )
        
        # 3. 평가 결과 분석 및 출력
        analyze_evaluation_results(result)
        
    except Exception as e:
        print(f"평가 중 오류 발생: {e}")